In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers


In [ ]:
tf.data.experimental.enable_debug_mode()
def process_image(image_path):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image , [224,224])
  image = tf.cast(image, tf.float32) / 255.0
  return image

***Loading the Dataset***

In [ ]:
augmented_dataset = tf.data.Dataset.list_files("/content/drive/MyDrive/Denoising_AutoEncoder/Train/augmented/*/*",shuffle=False)
augmented_dataset = augmented_dataset.map(process_image)
augmented_train_dataset = augmented_dataset.take(int(0.8 * len(augmented_dataset)))
augmented_val_dataset = augmented_dataset.skip(int(0.8 * len(augmented_dataset)))
clean_dataset = tf.data.Dataset.list_files("/content/drive/MyDrive/Denoising_AutoEncoder/Train/original/*/*",shuffle=False)
clean_dataset = clean_dataset.map(process_image)
clean_train_dataset = clean_dataset.take(int(0.8 * len(clean_dataset)))
clean_val_dataset = clean_dataset.skip(int(0.8 * len(clean_dataset)))

train_set = tf.data.Dataset.zip(augmented_train_dataset,clean_train_dataset).shuffle(5, seed=123).repeat(3).batch(32)
val_set = tf.data.Dataset.zip(augmented_val_dataset,clean_val_dataset).batch(32)

***Use SSIM and PSNR Metrics to evaluate the accuracy of the model***

In [ ]:
def SSIM(y_true, y_pred):
    return tf.image.ssim(y_true, y_pred, max_val=1.0)

def PSNR(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

***Buiding the Encoder-Decoder***

In [ ]:
def get_autoencoder(input_shape=(224,224,3), num_layers = 4, num_filters = 64):

  input = layers.Input(shape=input_shape)
  x = input

  # Encoder
  for i in range(num_layers):
    x = layers.Conv2D(num_filters, (3, 3), activation="relu", padding="same")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(num_filters, (3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)
    num_filters = num_filters * 2

  x = layers.Dropout(0.2)(x)

  # Decoder
  for i in range(num_layers):
    num_filters = num_filters //2
    x = layers.Conv2DTranspose(num_filters, (3, 3), strides=2, activation="relu", padding="same")(x)

  x = layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)
  # Autoencoder
  autoencoder = Model(input, x, name="autoencoder")

  initial_learning_rate = 0.001
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100,
    decay_rate=0.80,
    staircase=False)

  autoencoder.compile(optimizer="adam", loss="binary_crossentropy",metrics=[PSNR, tf.keras.metrics.RootMeanSquaredError(name="rmse"), SSIM])
  return autoencoder

In [ ]:
ae = get_autoencoder(num_layers=4)
ae.summary()

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_set = train_set.prefetch(buffer_size=AUTOTUNE)
val_set = val_set.prefetch(buffer_size=AUTOTUNE)
history = ae.fit(train_set , epochs=40 ,validation_data= val_set)
ae.save('path_to_save_the_autoencoder')

***Prediction***

In [ ]:
DAE = tf.keras.models.load_model('path_to_load_the_autoencoder')
image = process_image("path_to_denoise_the_signature'")
image = np.expand_dims(image , axis=0)
print(image.shape)
res = DAE.predict(image)